In [1]:
import pdbufr
import sys
import traceback
 
from eccodes import *
from ecmwf.opendata import Client
from math import isnan

from ipywidgets import interact
import os
import birdy
import geopandas as gpd
import pandas as pd
from datetime import datetime, timedelta
import ipyleaflet
import ipywidgets as widgets

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Download the Tropical Cyclone tracks from ECMWF's 00UTC ENS forecast
'''client.retrieve(
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target="track_data/ens_tracks.bufr",
)''';

In [3]:
## PICK STARTING DATE OF THE FORECAST ##

start_date_forecast = widgets.DatePicker(
    description = 'Forecast date:',
    value = datetime(datetime.now().year, datetime.now().month, datetime.now().day),
)

start_date_forecast.style.description_width = '90px'

# Print widget
start_date_forecast

DatePicker(value=datetime.datetime(2023, 7, 31, 0, 0), description='Forecast date:', step=1, style=Description…

In [4]:
## DATA DOWNLOAD AND OPEN AS DATAFRAME##
client = Client(source="azure")

client.retrieve(
    date=int(start_date_forecast.value.strftime("%Y%m%d")),
    time=0,
    stream="enfo",
    type="tf",
    step=240,
    target="track_data/tc_test.bufr",
);

# Load bufr file
def create_cycl_df():
    df_tracks = pdbufr.read_bufr("track_data/tc_test.bufr",
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "pressureReducedToMeanSeaLevel"))
    df1 = pdbufr.read_bufr("track_data/tc_test.bufr",
        columns=("stormIdentifier", "ensembleMemberNumber", "latitude", "longitude",
                 "windSpeedAt10M"))
    df_tracks["windSpeedAt10M"] = df1.windSpeedAt10M
    drop_condition = df_tracks.stormIdentifier < '11'
    df_tracks = df_tracks[drop_condition]
    return df_tracks

df_tracks = create_cycl_df()
storms = df_tracks.stormIdentifier.unique()

20230731000000-240h-enfo-tf.bufr:   0%|          | 0.00/651k [00:00<?, ?B/s]

In [5]:
## SELECT CYCLONE TO PLOT ##
cyclone = widgets.Dropdown(
    options = storms.tolist(),
    description = 'Active Storms:',
    disabled=False,
)
cyclone.style.description_width = '90px'

# Link cyclones data to forecast date selected
def update_forecast_data(_):
    client = Client(source="azure")
    client.retrieve(
        date=int(start_date_forecast.value.strftime("%Y%m%d")),
        time=0,
        stream="enfo",
        type="tf",
        step=240,
        target="track_data/tc_test.bufr",
    );
    df_tracks = create_cycl_df()
    storms = df_tracks.stormIdentifier.unique()
    cyclone.options = storms.tolist()
start_date_forecast.observe(update_forecast_data, names='value')

# Print widget
cyclone

Dropdown(description='Active Storms:', options=('08W', '70W', '71W', '72W', '73W', '74W', '75W', '76W', '77W',…

In [6]:
## SELECT ENSEMBLE MEMBER TO PLOT ##
df_cycl = df_tracks[df_tracks.stormIdentifier == cyclone.value]
members = df_cycl.ensembleMemberNumber.unique()

# Create widget for ensemble member selection
member = widgets.Dropdown(
    options = members.tolist(),
    value = 1,
    description = 'Ensemble member:',
    disabled=False,
)

# Link ensemble selection widget to cyclone selection widget
def update_ensemble_members(_):
    df_tracks = create_cycl_df()
    df_cycl = df_tracks[df_tracks.stormIdentifier == cyclone.value]
    members = df_cycl.ensembleMemberNumber.unique()
    member.options = members.tolist()

cyclone.observe(update_ensemble_members, names='value')

member.style.description_width = '120px'

# Print widget
member

Dropdown(description='Ensemble member:', options=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 1…

In [7]:
## TRACK DATAFRAME CREATION ##
def create_track_df(df_cycl, start_date, ens_member):
    df_track = df_cycl[df_cycl.ensembleMemberNumber == ens_member]
    
    # Add column of dates to the dataframe
    df_track.reset_index(inplace=True)
    df_track.drop(columns="index", inplace=True)
    df_track["date"] = start_date + (df_track.index.to_numpy()+1) * timedelta(hours=6)
    
    # Drop NaN in latitude and longitude columns
    df_track.dropna(subset = ['latitude', 'longitude'], inplace=True)
    
    # Locations and dates list for temporal horizon widget
    lat = df_track.latitude.values
    lon = df_track.longitude.values
    locations = [[lat[i], lon[i]] for i in range(len(lat))]
    track_dates = df_track["date"]
    
    return locations, track_dates

locations, track_dates = create_track_df(df_cycl, start_date_forecast.value, member.value)

In [8]:
## TEMPORAL HORIZON RANGE SLIDEBAR WITH DATES ##
options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in track_dates]
index = (0, len(options)-1)

temporal_horizon_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    # description='Temporal Horizon:',
    description='Define horizon:',
    orientation='horizontal',
    layout={'width': '500px'},
    disabled=False,
)

# Define the size of the descprition box of the slider and other styles
temporal_horizon_slider.style.description_width = '100px'
temporal_horizon_slider.style.handle_color = 'orange'

def print_date_range(date_range):
    start, end = date_range
    s = start.strftime('%d %b %Y %H:%M')
    e = end.strftime('%d %b %Y %H:%M')
    print(f'Temporal horizon: {s} - {e}')
    
# Link horizon slider to ensemble member selection
def update_horizon_ens(_):
    df_tracks = create_cycl_df()
    df_cycl = df_tracks[df_tracks.stormIdentifier == cyclone.value]
    locations, track_dates = create_track_df(df_cycl, start_date_forecast.value, member.value)
    options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in track_dates]
    temporal_horizon_slider.options = options
    temporal_horizon_slider.index = (0, len(options)-1)

cyclone.observe(update_horizon_ens, names='value')
member.observe(update_horizon_ens, names='value')
start_date_forecast.observe(update_horizon_ens, names='value')

# Print widget with selected temporal horizon displayed
temporal_horizon_slider
widgets.interact(
        print_date_range,
        date_range=temporal_horizon_slider
    );

interactive(children=(SelectionRangeSlider(description='Define horizon:', index=(0, 40), layout=Layout(width='…

In [9]:
## TEMPORAL SLIDEBAR TO SELECT SNAPSHOT##
# Function to convert datetime to integer
def datetime_to_int(dt):
    return int((dt - start_date_forecast.value - timedelta(hours=6)).total_seconds() // timedelta(hours=6).total_seconds())

# Function to convert interger to datetime
def int_to_datetime(ts):
    return start_date_forecast.value + timedelta(hours=6) + ts * timedelta(hours=6)

timestep_slider=widgets.IntSlider(
    min=0, 
    max=datetime_to_int(track_dates.iloc[-1].to_pydatetime()), 
    step=1, 
    value=0, 
    description='Timestep:')

# Increase the size of the descprition box of the slider and other styles
timestep_slider.style.description_width = '65px'
timestep_slider.style.handle_color = 'orange'
    
def print_timestep(dt):
    date = int_to_datetime(dt)
    print('Selected date: ', date.strftime('%d %b %Y %H:%M'))
    
# Link the timestep selection slider to the horizon slider
def update_snapshot_slider(_):
    start, end = temporal_horizon_slider.value
    timestep_slider.value = datetime_to_int(start.to_pydatetime())
    timestep_slider.min = datetime_to_int(start.to_pydatetime())
    timestep_slider.max = datetime_to_int(end.to_pydatetime())
temporal_horizon_slider.observe(update_snapshot_slider, names='value')

# Print widget with selected date displayed
timestep_slider
widgets.interact(
    print_timestep,
    dt=timestep_slider
);

interactive(children=(IntSlider(value=0, description='Timestep:', max=40, style=SliderStyle(description_width=…

In [10]:
## INTERACTIVE MAP FOR TROPICAL CYCLONE TRACK ##
# Create the basemap for plotting
initial_lat_lon = (locations[0][0], locations[0][1])
tc_track_map = ipyleaflet.Map(
    center=initial_lat_lon,
    basemap=ipyleaflet.basemaps.OpenStreetMap.France,
    zoom = 3.5,
)

# Add a custom zoom slider
# zoom_slider = widgets.IntSlider(description="Zoom level:", min=1, max=17, value=4)
# widgets.jslink((zoom_slider, "value"), (tc_track_map, "zoom"))
# widget_control1 = ipyleaflet.WidgetControl(widget=zoom_slider, position="topright")
# tc_track_map.add_control(widget_control1)

# Add tropical cyclone track to the map
track = ipyleaflet.Polyline(
    locations=locations,
    color="blue" ,
    fill=False,
    name='Track',
)

# Link the cyclone track to the ensemble number and the temporal horizon 
def update_track(_):
    df_tracks = create_cycl_df()
    df_cycl = df_tracks[df_tracks.stormIdentifier == cyclone.value]
    locations, track_dates = create_track_df(df_cycl, start_date_forecast.value, member.value)
    start, end = temporal_horizon_slider.value
    start_index = datetime_to_int(start.to_pydatetime())
    end_index = datetime_to_int(end.to_pydatetime())
    track.locations = locations[start_index:end_index+1]

cyclone.observe(update_track, names='value')
member.observe(update_track, names='value')
temporal_horizon_slider.observe(update_track, names='value')
start_date_forecast.observe(update_track, names='value')

tc_track_map.add_layer(track)

# Add a marker to the map, for the snapshot selected
marker = ipyleaflet.CircleMarker(location=locations[timestep_slider.value],
                                 draggable=False,
                                 name = 'Timestep',
                                 color = "orange",
                                 fill_color = "orange",
                                 radius = 2,
                                )

# Link the marker to the snapshot selections slider
def update_marker_location(_):
    df_tracks = create_cycl_df()
    df_cycl = df_tracks[df_tracks.stormIdentifier == cyclone.value]
    locations, track_dates = create_track_df(df_cycl, start_date_forecast.value, member.value)
    marker.location = locations[timestep_slider.value]

cyclone.observe(update_marker_location, names='value') 
member.observe(update_marker_location, names='value')
timestep_slider.observe(update_marker_location, names='value')
start_date_forecast.observe(update_marker_location, names='value')

tc_track_map.add_layer(marker)

# Add layers widget to the map
tc_track_map.add_control(ipyleaflet.LayersControl());

# Print map
tc_track_map

Map(center=[21.900000000000002, 131.8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_ti…

In [11]:
locations, track_dates = create_track_df(df_cycl, start_date_forecast.value, member.value)
options = [(timestep.strftime('%d/%m %H'), timestep) for timestep in track_dates]

In [12]:
df_track = df_cycl[df_cycl.ensembleMemberNumber == member.value]
    
# Add column of dates to the dataframe
df_track.reset_index(inplace=True)
df_track.drop(columns="index", inplace=True)
df_track["date"] = start_date_forecast.value + (df_track.index.to_numpy()+1) * timedelta(hours=6)

# Drop NaN in latitude and longitude columns
df_track.dropna(subset = ['latitude', 'longitude'], inplace=True)

# Locations and dates list for temporal horizon widget
lat = df_track.latitude.values
lon = df_track.longitude.values
locations = [[lat[i], lon[i]] for i in range(len(lat))]
track_dates = df_track["date"]